<div class="alert alert-block alert-warning">Some parts of this tutorial may not work properly when viewing the tutorial on github. (In particular plotting.) To get the most out of this tutorial, consider running the tutorial locally in your own clone of the repository as explained in the [README](https://github.com/katelaan/sloth/blob/master/README.md). This will also allow you to modify the code examples.</div>

# `sloth` Tutorial

`sloth` is a solver for the separation logic $\mathbf{SL}_\mathsf{data}^{*}$. `sloth` comes both with a command-line interface (CLI) and with a Python API. This tutorial focuses on the use of the API, but also gives an overview of the SMT-LIB extension that serves as the input format for the CLI.

I assume that you have seen separation logic before, though not necessarily $\mathbf{SL}_\mathsf{data}^{*}$. I will explain some more unusual features of $\mathbf{SL}_\mathsf{data}^{*}$ as we go along.

## The `sloth` Python API: Create Formulas, Solve Formulas, and Visualize Models

The `sloth` API allows you to build $\mathbf{SL}_\mathsf{data}^{*}$ formulas and combine them with `Z3` expressions as built through `Z3`'s Python API.

We begin by importing some useful functions from `Z3` as well as the `sloth` API.

In [1]:
from z3 import And, Or, Not, Ints
from sloth import *

The easiest way to understand $\mathbf{SL}_\mathsf{data}^{*}$ and `sloth` is to build $\mathbf{SL}_\mathsf{data}^{*}$ formulas via the API, check them for satisfiability and inspect the models that z3 finds. Let's begin with the equivalent API call for the $\mathbf{SL}_\mathsf{data}^{*}$ formula $\mathsf{list}(x)$, a null-terminated list rooted in `x`.

In [2]:
sl.list('x')

sl.list(x)

As you can see, we prefix the `list` call with `sl.` Notice that the quotes around `x` are gone in the output. This is because the function call has returned a `Z3.ExprRef` instance. I will explain later why we reuse `Z3`'s expressions in our API. But let's first do something with the above formula.
    

In [3]:
is_sat(sl.list('x'))

True

In [4]:
model(sl.list('x'))

Model [
  Struct sl.list [
    locs = Integers(0, 1:[x])
    null = 0
    data = undefined
    next = 1->0
    footprints:
    _Xnext=[1], _Xdata=[1]
  ]
  Data [undefined]
]

The formula is satisfiable and `sloth` has found a model: A list of length 1 from x to null.

Let's examine a few more features: Allocating individual pointers, list segments, and the combination via the separating conjunction:

In [5]:
m = model(sl.sepcon(sl.list.seg('x', 'y'), sl.list.next('y', sl.list.null))); m

Model [
  Struct sl.list [
    locs = Integers(0:[y], 1:[x], 3:[sl.list.null])
    null = 3
    data = undefined
    next = 0->3, 1->0
    footprints:
    _Xnext=[0, 1], _Xdata=[1]
  ]
  Data [undefined]
]

Observe that `x` and `y` are interpreted as different locations, since they are both allocated and separated by a separating conjunction. We can use the API to visualize the model:

In [6]:
iplot(m)

<div class="alert alert-block alert-warning">If you don't see the visualization, you're probably viewing the tutorial on github rather than locally. Consider following the steps in the [README](https://github.com/katelaan/sloth/blob/master/README.md) to run the tutorial locally. This will also allow you to modify the code examples.</div>

$\mathbf{SL}_\mathsf{data}^{*}$ also supports trees and tree segments. For example, we can define a tree segment rooted in `t` with two non-null leaves `u` and `v` that have backpointers to the root originating in leaves `u` and `v`. The `.seg2` suffix of the tree predicate call denotes a tree segment with 2 stop nodes.

In [7]:
model(sl.sepcon(sl.tree.seg2('t', 'u', 'v'), sl.tree.left('u', 't'), sl.tree.right('v', 't')))

Model [
  Struct sl.tree [
    locs = Integers(0, 1:[u], 2:[v], 3, 4, 5, 6:[t])
    null = 0
    data = undefined
    right = 2->6, 3->0, 4->3, 5->4, 6->5
    left = 1->6, 3->2, 4->0, 5->0, 6->1
    footprints:
    _Xleft=[1, 3, 4, 5, 6], _Xright=[2, 3, 4, 5, 6], _Xdata=[3, 4, 5, 6]
  ]
  Data [undefined]
]

In this manner, we can define cycles as well as bounded sharing in the heap, even though both $\ls$ and $\tree$ predicates themselves always describe acyclic structures.

$\mathbf{SL}_\mathsf{data}^{*}$ formulas can also constrain the data stored in data structures by means of universal data constraints. But before we look at that, let's allocate a few appropriately-sorted variables to spare ourselves some typing for the rest of the tutorial.

In [8]:
t, u, v = sl.tree.locs('t u v')
x, y, z, w = sl.list.locs('x y z w')
d, e, f, g = Ints('d e f g')

Now let's specify sorted lists rooted in `x`! In $\mathbf{SL}_\mathsf{data}^{*}$, we write this as $\mathsf{list}(x, (\textsf{n}, \alpha < \beta))$. Intuitively, the data predicate $\alpha < \beta$ holds for all locations in the list rooted in `x`. $\alpha$ and $\beta$ are special variables that range over the data stored in the list locations. In the above example, the data predicate specifies that for all locations $\ell_1$ containing data value $\alpha$ and $\ell_2$ containing data value $\beta$, if $\ell_2$ is reachable from $\ell_1$ by first taking an $\textsf{n}$-pointer, then $\alpha < \beta$ holds.

While this is somewhat complicated, the idea is very simple: Look at the list. Fix two arbitrary locations. Then the one later in the list stores a larger data value than the one earlier in the list. In other words, the list is sorted.

In `sloth`, we can specify the sorted list with a very similar syntax. Note the additional `dpred`, though. This signifies that we're defining a data predicate rather than an ordinary list predicate. The `.next` corresponds to the $\textsf{n}$ in $\ls(x, (\textsf{n}, \alpha < \beta))$. (It will become clear why it makes sense to specify a field when we're defining data predicates for trees.)

In [9]:
sorted_list = sl.list.dpred.next(sl.alpha < sl.beta, x)
model(sorted_list)

Model [
  Struct sl.list [
    locs = Integers(0, 1:[x])
    null = 0
    data = undefined
    next = 1->0
    footprints:
    _Xdata=[1], _Xnext=[1]
  ]
  Data [undefined]
]

Hurray, `sloth` found a sorted list! Which contains exactly one element! As you can see, Z3 did not even define the data function (`data = undefined`). This sometimes happens when the truth value of the formula does not depend on the interpretation of a function. This is the case here: No matter the data it contains, a list of length 1 is indeed sorted.

So let's make things more interesting and force a minimum length. We will use an auxiliary function that makes it less cumbersome to specify data structures of fixed size.

In [10]:
from sloth.slbuilders import list_of_length
length8 = list_of_length(8, x, sl.list.null); length8

sl.sepcon(sl.sepcon(sl.sepcon(sl.sepcon(sl.list.next(x, a1),
                                        sl.list.next(a1, a2)),
                              sl.sepcon(sl.list.next(a2, a3),
                                        sl.list.next(a3, a4))),
                    sl.sepcon(sl.sepcon(sl.list.next(a4, a5),
                                        sl.list.next(a5, a6)),
                              sl.sepcon(sl.list.next(a6, a7),
                                        sl.list.next(a7,
                                        sl.list.null)))),
          sl.sepcon(sl.sepcon(sl.sepcon(sl.list.data(x, d0),
                                        sl.list.data(a1, d1)),
                              sl.sepcon(sl.list.data(a2, d2),
                                        sl.list.data(a3, d3))),
                    sl.sepcon(sl.sepcon(sl.list.data(a4, d4),
                                        sl.list.data(a5, d5)),
                              sl.sepcon(sl.list.data(a6, d6),
           

It would have been kind of annoying to write that by hand, wouldn't it?

Next we simply conjoin `length8` and `sorted_list`. This is where it is really useful that the `sloth` API constructs `Z3` expression references: We can use all the features of the Z3 API together with our SL assertions. You can construct the conjunction of two $\mathbf{SL}_\mathsf{data}^{*}$ formulas by calling `Z3`'s `And` function.

In [11]:
sorted_length8 = And(sorted_list, length8)
m = model(sorted_length8); m

Model [
  Struct sl.list [
    locs = Integers(7:[x], 15:[a1], 16:[a2], 17:[a3], 18:[a4], 19:[a5], 20:[a6], 21:[a7], 22:[sl.list.null])
    null = 22
    data = 7->-5, 15->-4, 16->-3, 17->-2, 18->-1, 19->0, 20->1, 21->2
    next = 7->15, 15->16, 16->17, 17->18, 18->19, 19->20, 20->21, 21->22
    footprints:
    _Xdata=[7, 15, 16, 17, 18, 19, 20, 21], _Xnext=[7, 15, 16, 17, 18, 19, 20
    21]
  ]
  Data [
    d0=-5, d1=-4, d2=-3, d3=-2, d4=-1, d5=0, d6=1, d7=2
  ]
]

Let's plot the model to make it easier to figure out what's going on here!

In [12]:
iplot(m)

As you can see, the data stored in the nodes really are sorted.

The per-field data predicates are really useful when we want to specify properties of trees. For example, the $\mathbf{SL}_\mathsf{data}^{*}$ formula $\mathsf{tree}(t, \{(\mathsf{l}, \alpha > \beta), (\mathsf{r}, \alpha < \beta)\}$ specifies a binary search tree: Whenever I get to a location by first taking the left ($\mathsf{l}$) field, the data value gets smaller; and whenever I go right first, the data value gets larger.

Let's express this in the `sloth` API. We need to assert the predicates separately. I.e., we say that the heap contains both a tree where the first predicate holds and a tree where the second predicate holds. $\mathbf{SL}_\mathsf{data}^{*}$ semantics of conjunction forces that the footprints of these trees coincide, so the predicates in fact hold in the same tree! To get an interesting model, we again force a minimal size by conjoining a formula that allocates a few nodes. 

In [13]:
from sloth.slbuilders import tree_of_size
size5 = tree_of_size(6, t)
binary_search_tree = And(sl.tree.dpred.left(sl.alpha > sl.beta, t),
                        sl.tree.dpred.right(sl.alpha < sl.beta, t))
m = model(And(size5, binary_search_tree)); m

Model [
  Struct sl.tree [
    locs = Integers(6:[a1], 7:[a2], 8:[a3], 9:[a4], 10:[a5], 11:[sl.tree.null], 12:[t])
    null = 11
    data = 6->-3, 7->1, 8->-4, 9->-2, 10->0, 12->-1
    right = 6->9, 7->11, 8->11, 9->11, 10->11, 12->7
    left = 6->8, 7->10, 8->11, 9->11, 10->11, 12->6
    footprints:
    _Xright=[6, 7, 8, 9, 10, 12], _Xleft=[6, 7, 8, 9, 10, 12], _Xdata=[6, 7, 8
    9, 10, 12]
  ]
  Data [
    d0=-1, d1=-3, d2=1, d3=-4, d4=-2, d5=0
  ]
]

In [14]:
iplot(m)

To make the plots somewhat easier to understand, tree edges to null are suppressed by default. You can have them added to the graph if you like.

In [15]:
iplot(m, draw_tree_edges_to_null=True)

## The Command-Line Interface & the `sloth` Input Format

CLI input files use a custom extension of SMT-LIB. Let's look at an example. (The `benchmark` function looks for the file `overlaid-four-nodes.smt2` in the (subdirectories of the) `benchmarks` folder of the repository. `show` reads the file and dumps its contents to stdout.)

In [16]:
benchmark('overlaid-four-nodes.smt2')

'benchmarks/list-boolean-closure/overlaid-four-nodes.smt2'

In [17]:
show(benchmark('overlaid-four-nodes.smt2'))

(declare-const x sl.list.loc)
(declare-const y sl.list.loc)
(declare-const z sl.list.loc)
(declare-const w sl.list.loc)
(declare-const d Int)
(declare-const e Int)
(declare-const f Int)
(declare-const g Int)
(assert (and (sl.list x)
             (sl.sepcon (sl.sepcon (sl.list.dpointsto x y d)
                              (sl.list.dpointsto y z e))
                   (sl.sepcon (sl.list.dpointsto z w f)
                              (sl.list.dpointsto w sl.list.null g)))))



Let's go through this example in detail.

- All sorts, functions etc. in our custom extension begin with the prefix `sl.`, usually followed by the data structure to which they apply, e.g. `list` or `tree`.
- As usual in SMT-LIB, all variables used must be declared before their use
- List locations are declared to be of sort `sl.list.loc`; tree locations of sort `sl.tree.loc`
- We assert a classical conjunction using the standard SMT-LIB `and`
- We assert that the heap consists (exclusively) of a list starting in a, `(sl.list a)`
- We assert that in the same heap (`and`), there are four separate allocated list nodes. Each node has a `next` and a `data` field. For example, `(sl.list.dpointsto a b e)` says that `a` is allocated, its next pointer is referenced by `b` and it's data field contains the value of the integer variable `e`.

You can invoke `sloth` on the benchmark by calling

```
$ ./sloth.sh benchmarks/list-boolean-closure/overlaid-four-nodes.smt2
```

But you can also parse the benchmark via the API:

In [18]:
expr = parse(benchmark('overlaid-four-nodes.smt2')); expr

ParseResult(expr=And(sl.list(x),
    sl.sepcon(sl.sepcon(sl.list.dpointsto(x, y, d),
                        sl.list.dpointsto(y, z, e)),
              sl.sepcon(sl.list.dpointsto(z, w, f),
                        sl.list.dpointsto(w,
                                        sl.list.null,
                                        g)))), backend=Integer locations & lambdas)

Ignore the part about the `backend`. What matters for now is that the parse result can serve as input for `encode`, `model`, `iplot` etc. just like any expression created via the API!

In [19]:
iplot(model(expr))

Currently, no full specification of the input language exists. The best way to understand the language is to have a look at benchmarks that interest you. As we saw above, you can simply load them into your Python session.

In [20]:
expr = parse(benchmark('overlaid-four-nodes.smt2'))
m = solve(expr); m

Model [
  Struct sl.list [
    locs = Integers(0:[x], 1:[y], 3:[z], 5:[w], 7:[sl.list.null])
    null = 7
    data = 0->2, 1->4, 3->6, 5->8
    next = 0->1, 1->3, 3->5, 5->7
    footprints:
    _Xnext=[0, 1, 3, 5], _Xdata=[0, 1, 3, 5]
  ]
  Data [
    d=2, e=4, f=6, g=8
  ]
]

## Advanced Features of the `sloth` API

### Graph Representations
Model plotting is based on a graph representation of the SMT model. The graph representation may be easier to understand than the result of `model` calls. You don't have to plot the model to get a hold of it:

In [21]:
gm = model_to_graph(m); gm

Graph({0, 1, 3, 5, 7}, {(0, 'data'): 2, (0, 'next'): 1, (1, 'data'): 4, (1, 'next'): 3, (3, 'data'): 6, (3, 'next'): 5, (5, 'data'): 8, (5, 'next'): 7}, {'sl.list.null': 7, 'w': 5, 'x': 0, 'y': 1, 'z': 3}, {'d': 2, 'e': 4, 'f': 6, 'g': 8})

Graph models have a rich API themselves. This can further aid understanding the model. Here are some examples:

In [22]:
{'1.) The next-pointer of x is allocated' : gm.is_alloced('x', 'next'),
  '2.) x points to the following location' : gm.resolve('x'),
  '3.) The next and data pointers of x point to' : gm.successors('x', ['next', 'data']), 
  '4.) w is reachable from x' : gm.reach(['next'], 'x', 'w'),
  '5.) z is reachable from w' : gm.reach(['next'], 'w', 'z')
}

{'1.) The next-pointer of x is allocated': True,
 '2.) x points to the following location': 0,
 '3.) The next and data pointers of x point to': [1, 2],
 '4.) w is reachable from x': True,
 '5.) z is reachable from w': False}

### Inspect the Encoding

It's possible to look at the SMT encoding that `sloth` generates. It contains comments, but is both large and complicated, so without studying the IJCAR paper, you will likely have a large time understanding what's going on.

In [23]:
z3input = encode(sl.list.seg(x, y))
print(z3input.to_smt2_string())

(declare-fun _X () (Array Int Bool))
(declare-fun _Xdata () (Array Int Bool))
(declare-fun _Xnext () (Array Int Bool))
(declare-fun _Y1data () (Array Int Bool))
(declare-fun _Y1next () (Array Int Bool))
(declare-fun _Z () (Array Int Bool))
(declare-fun _x0 () Int)
(declare-fun _x1 () Int)
(declare-fun _x2 () Int)
(declare-fun sl.list.null () Int)
(declare-fun x () Int)
(declare-fun y () Int)
(declare-fun _r_1 (Int Int) Bool)
(declare-fun _r_2 (Int Int) Bool)
(declare-fun _r_3 (Int Int) Bool)
(declare-fun sl.list.data (Int) Int)
(declare-fun sl.list.next (Int) Int)
(assert
  (and
    ;; ***** A (bound: 3) *****
    (and
      ;; Placing (sl.list.seg x y) in the global context
      (and
        ;; Structural encoding of list(x, [y]) of size 3 with data constraints None
        (and
          ;; structure: _Z : SET(Int) is an acyclic data structure rooted in x
          (and
            ;; The root is in _Z : SET(Int)
            (=>
              (not
                ;; x is a stop node

It's perhaps more useful to write the encoding to a file that you can inspect in an editor and also directly feed to `Z3` if you like.

In [24]:
z3input.to_file('encoding.smt2', verbose = True)

Wrote encoding to "encoding.smt2".
